# ! ALWAYS CLEAR OUTPUT BEFORE COMMIT

## Imports

In [62]:
import pandas as pd
import numpy as np
import yaml
import os
import re

## Utils

In [63]:
def sanitize_llm_output(raw_content):
    """
    Strips Markdown code blocks (```yaml ... ```) from LLM output.
    """
    # Strip whitespace first
    content = raw_content.strip()
    
    # Regex to capture content inside ``` blocks
    pattern = r"```(?:yaml|yml)?\n(.*?)```"
    match = re.search(pattern, content, re.DOTALL)
    
    if match:
        # Return the captured group inside the backticks
        return match.group(1).strip()
    
    # If no code blocks found, return original (assuming it's raw YAML)
    return content

def sanitize_charge_amount(raw_content):
    """
    Sanitizes the charge amount by removing any non-numeric characters except for '.'.
    """
    # Use regex to remove unwanted characters
    sanitized = re.sub(r"[^0-9.]", "", raw_content)

    # to number format
    try:
        float(sanitized)
    except ValueError:
        sanitized = "nan"
    return sanitized

## Data

### Get ground truth data

In [64]:

# print current working directory
print("Current Working Directory:", os.getcwd())
ground_truth_dir_path = "safe/ground_truth_bank_statement"

# parse yaml files to a custom data structure
transaction_item = {
    "posting_date": str,
    "transaction_date": str,
    "description": str,
    "reference": str,
    "mcc": str,
    "charge": str,
}
ground_truth_data_all = []
for filename in os.listdir(ground_truth_dir_path):
    if filename.endswith(".yaml"):
        ground_truth_data = []
        print(f"Processing file: {filename}")
        file_path = os.path.join(ground_truth_dir_path, filename)
        try:
            with open(file_path, 'r') as file:
                yaml_data = yaml.safe_load(file)
                for item in yaml_data:
                    transaction = {key: item.get(key, "") for key in transaction_item.keys()}
                    # sanitize charge amount
                    transaction["charge"] = sanitize_charge_amount(transaction["charge"])
                    ground_truth_data.append(transaction)
            ground_truth_data_all.append(ground_truth_data)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# to dataframe
ground_truth_df = pd.DataFrame(ground_truth_data_all[0])
print(ground_truth_df.head())

Current Working Directory: /home/polski/capstone_proj/senior_design/src/backend
Processing file: november.yaml
Processing file: october.yaml
Processing file: august.yaml
Processing file: december.yaml
  posting_date transaction_date                               description  \
0        11/13            11/12    AMZN Mktp US*WB9OW3RL3 Amzn.com/billWA   
1        11/15            11/14  TOP GOLF LOUISVILLE 64-3 866-867-4653 KY   
2        11/16            11/15                 KROGER #356 MIDDLETOWN KY   
3        11/16            11/15              CKITCHEN COM 718-338-5800 NY   
4        11/17            11/16                 KROGER #356 MIDDLETOWN KY   

                 reference   mcc   charge  
0  24692163316103100975838  5942   169.72  
1  24431063318700451520893  7999   112.00  
2  24445713319300361413611  5411    68.05  
3  24009583319300746515036  5046  1827.00  
4  24445713320300357682895  5411    83.89  


In [65]:
"""
Processing file[0]: november.yaml
Processing file[1]: october.yaml
Processing file[2]: august.yaml
Processing file[3]: december.yaml
"""
# set ground truth dataframe for comparison
ground_truth_df = pd.DataFrame(ground_truth_data_all[0])
ground_truth_df

,posting_date,transaction_date,description,reference,mcc,charge
0,11/13,11/12,AMZN Mktp US*WB9OW3RL3 Amzn.com/billWA,24692163316103100975838,5942,169.72
1,11/15,11/14,TOP GOLF LOUISVILLE 64-3 866-867-4653 KY,24431063318700451520893,7999,112.00
2,11/16,11/15,KROGER #356 MIDDLETOWN KY,24445713319300361413611,5411,68.05
3,11/16,11/15,CKITCHEN COM 718-338-5800 NY,24009583319300746515036,5046,1827.00
4,11/17,11/16,KROGER #356 MIDDLETOWN KY,24445713320300357682895,5411,83.89
5,11/17,11/17,AMZN Mktp US*J00GZ02L3 Amzn.com/billWA,24692163321106675425652,5942,31.79
6,11/17,11/17,AMZN Mktp US*CE4J77CQ3 Amzn.com/billWA,24692163321106688639539,5942,26.49
7,11/20,11/16,MCMASTER-CARR 630-834-9600 IL,24789303321705700168272,5085,644.56
8,11/20,11/17,TEXAS ROADHOUSE #2498 LOUISVILLE KY,24231683322400000500063,5812,232.10
9,11/21,11/20,KROGER #309 LOUISVILLE KY,24445713324300373765522,5411,11.99


### Get parsed data

In [66]:
import pdfplumber

def validate_pdf(path):
  """
  Validate a PDF file by checking its contents.

  Args:
    path: The path to the PDF file.

  Raises:
    ValueError: If the PDF is empty or contains only text.
    ValueError: If the PDF contains embedded images.

  Returns:
    None
  """
  with pdfplumber.open(path) as pdf:
    if len(pdf.pages) == 0:
        raise ValueError("Empty PDF")

    # Reject scanned PDFs
    text = pdf.pages[0].extract_text()
    if not text or len(text.strip()) < 100:
        raise ValueError("Likely scanned or image-only PDF")

    # # Optional: check for embedded images dominance
    # if len(pdf.pages[0].images) > 5:
    #     raise ValueError("Image-heavy PDF rejected")

def detect_bank(pdf):
  """
  Detect the bank in a PDF file by analyzing its first page.

  Args:
    pdf: The PDF object.

  Returns:
    A string representing the detected bank.
  """
  first_page_text = pdf.pages[0].extract_text()

  if "CHASE" in first_page_text:
      return "CHASE_V1"
  if "BANK OF AMERICA" in first_page_text:
      return "BOA_V1"

  raise ValueError("Unknown bank template")

def get_table1_bbox(page):
  text = page.extract_text()

  if "MCC" not in text:
      return None

  # Manually tuned bounding box per template
  return (15, 370, 590, 550) # (x0, y0, x1, y1)

def get_table2_bbox(page):
  text = page.extract_text()

  if "MCC" not in text:
      return None

  # Manually tuned bounding box per template
  return (15, 85, 590, 790) # (x0, y0, x1, y1)

def extract_transactions(pdf, table_settings):
  transactions = []

  # for page in pdf.pages:
  #     bbox = get_table_bbox(page)
  #     cropped = page.crop(bbox)

  #     table = cropped.extract_table(table_settings)

  #     if not table:
  #         raise ValueError("No table extracted")

  #     for row in table:
  #         rows.append(row)

  # return rows
  for page in pdf.pages:
    print(f'Processing page {page.page_number}')
    if get_table1_bbox(page) and page.page_number == 1:
      print(f'Found transaction table in page {page.page_number} using table1_bbox')
      bbox = get_table1_bbox(page)
      cropped = page.crop(bbox)
      extraced_text = cropped.extract_text()
      print(extraced_text)
      transactions.append(extraced_text)
    elif get_table2_bbox(page) and page.page_number == 3:
      print(f'Found transaction table in page {page.page_number} using table2_bbox')
      bbox = get_table2_bbox(page)
      cropped = page.crop(bbox)
      extraced_text = cropped.extract_text()
      print(extraced_text)
      transactions.append(extraced_text)
    else:
      continue
  return transactions


In [67]:
def parse_statement(path):
  validate_pdf(path)

  with pdfplumber.open(path) as pdf:
    bank = detect_bank(pdf)

    # if bank == "BOA_V1":
    #     settings = CHASE_TABLE_SETTINGS
    # else:
    #     raise ValueError("Unsupported bank")
    settings = 2
    raw_rows = extract_transactions(pdf, settings)

    # txns = [parse_row(r) for r in raw_rows if r[0] != "Date"]
    # validate_transactions(txns)

    return raw_rows

# sample
result = parse_statement("safe/sample/statement_nov.pdf")
print(result)

# pick ground truth dataframe for comparison
"""
Processing file[0]: november.yaml
Processing file[1]: october.yaml
Processing file[2]: august.yaml
Processing file[3]: december.yaml
"""
ground_truth_df = pd.DataFrame(ground_truth_data_all[0])


Processing page 1
Found transaction table in page 1 using table1_bbox
Transactions
PostingTransaction
Date Date Description ReferenceNumber MCC Charge Credit
11/13 11/12 AMZN Mktp US*WB9OW3RL3 Amzn.com/billWA 24692163316103100975838 5942 169.72
11/15 11/14 TOP GOLF LOUISVILLE 64-3 866-867-4653 KY 24431063318700451520893 7999 112.00
11/16 11/15 KROGER #356 MIDDLETOWN KY 24445713319300361413611 5411 68.05
11/16 11/15 CKITCHEN COM 718-338-5800 NY 24009583319300746515036 5046 1,827.00
11/17 11/16 KROGER #356 MIDDLETOWN KY 24445713320300357682895 5411 83.89
11/17 11/17 AMZN Mktp US*J00GZ02L3 Amzn.com/billWA 24692163321106675425652 5942 31.79
11/17 11/17 AMZN Mktp US*CE4J77CQ3 Amzn.com/billWA 24692163321106688639539 5942 26.49
11/20 11/16 MCMASTER-CARR 630-834-9600 IL 24789303321705700168272 5085 644.56
11/20 11/17 TEXAS ROADHOUSE #2498 LOUISVILLE KY 24231683322400000500063 5812 232.10
11/21 11/20 KROGER #309 LOUISVILLE KY 24445713324300373765522 5411 11.99
11/22 11/20 TOP GOLF LOUISVILLE 64

In [68]:
import pandas as pd


raw_text = '\n '.join(result)
# print(raw_text)

# 2. Clean and split the text into individual lines
lines = raw_text.strip().split('\n')

data_rows = []

# 3. Iterate through lines to parse them
for line in lines:
    line = line.strip()
    
    
    # If line does not start with a number then skip
    if not line or not line[0].isdigit():
      continue
    # Split the line by whitespace
    parts = line.split()
    
    # Validation: Ensure we have enough columns
    # Minimum required: 2 dates + 1 description + 1 ref + 1 mcc + 1 charge = 6 items
    if len(parts) >= 6:
        # Index Mapping:
        # 0: Posting Date
        # 1: Transaction Date
        # 2 to len-3: Description (join the middle parts)
        # len-3: Reference Number
        # len-2: MCC
        # len-1: Charge
        
        row = {
            'posting_date': parts[0],
            'transaction_date': parts[1],
            'description': ' '.join(parts[2:-3]),
            'reference': parts[-3].strip(),
            'mcc': parts[-2].strip(),
            'charge': sanitize_charge_amount(parts[-1].strip())
        }
        data_rows.append(row)

# 4. Create the DataFrame
df = pd.DataFrame(data_rows)

# # Optional: Convert date columns to datetime objects for better manipulation
# df['posting_date'] = pd.to_datetime(df['posting_date'])
# df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Display the result
print(df.to_string(index=False))
print(len(df))


parsed_df = df

posting_date transaction_date                              description               reference  mcc  charge
       11/13            11/12   AMZN Mktp US*WB9OW3RL3 Amzn.com/billWA 24692163316103100975838 5942  169.72
       11/15            11/14 TOP GOLF LOUISVILLE 64-3 866-867-4653 KY 24431063318700451520893 7999  112.00
       11/16            11/15                KROGER #356 MIDDLETOWN KY 24445713319300361413611 5411   68.05
       11/16            11/15             CKITCHEN COM 718-338-5800 NY 24009583319300746515036 5046 1827.00
       11/17            11/16                KROGER #356 MIDDLETOWN KY 24445713320300357682895 5411   83.89
       11/17            11/17   AMZN Mktp US*J00GZ02L3 Amzn.com/billWA 24692163321106675425652 5942   31.79
       11/17            11/17   AMZN Mktp US*CE4J77CQ3 Amzn.com/billWA 24692163321106688639539 5942   26.49
       11/20            11/16            MCMASTER-CARR 630-834-9600 IL 24789303321705700168272 5085  644.56
       11/20            11/1

In [69]:
# # parsed_data_path = "safe/generated_responses/parsed_bank_statement_colab_8BFP8.yaml"
# # parsed_data_path = "safe/generated_responses/parsed_bank_statement_nov_local_4B.yaml"
# # parsed_data_path = "safe/generated_responses/parsed_bank_statement_nov_colab_8BFP8_modified.yaml"
# parsed_data_path ="safe/generated_responses/parsed_bank_statement_nov_local_deterministic_8BFP4.yaml"


# cleaned_data_list = []

# try:
#     with open(parsed_data_path, 'r') as file:
#         print(f"Parsing file: {parsed_data_path}")
#         raw_content = file.read()
        
#         # Sanitize the string
#         yaml_content = sanitize_llm_output(raw_content)
        
#         # Parse YAML string to Python List/Dict
#         # yaml.safe_load handles the conversion
#         parsed_raw_data = yaml.safe_load(yaml_content)
        
#         if parsed_raw_data is None:
#             parsed_raw_data = [] # Handle empty files
            

#     for item in parsed_raw_data:
#         # Ensure item is a dict before processing
#         if isinstance(item, dict):
#             # Normalization: Use .get() to fill missing keys with empty strings
#             # If you don't have a specific schema, just use 'item' directly
#             transaction = {key: item.get(key, "") for key in transaction_item.keys()}
#             # Sanitize the charge amount
#             transaction["charge"] = sanitize_charge_amount(transaction.get("charge", ""))
#             cleaned_data_list.append(transaction)
            
# except yaml.YAMLError as exc:
#     print(f"Error parsing YAML: {exc}")
# except Exception as e:
#     print(f"An error occurred: {e}")

# # to df
# parsed_df = pd.DataFrame(cleaned_data_list)
# print(parsed_df)

## Compare ground truth data and parsed data

In [70]:
# compare parsed item count to ground truth
print(f"Ground Truth Count: {len(ground_truth_df)}, Parsed Count: {len(parsed_df)}")
if len(ground_truth_df) != len(parsed_df):
    print("(!) Mismatch in number of transactions between ground truth and parsed data.")
else:
    print("(*) Transaction counts match between ground truth and parsed data.")

    # check for any differences in reference numbers
    gt_references = set(ground_truth_df['reference'].str.strip())
    parsed_references = set(parsed_df['reference'].str.strip()) 
    missing_in_parsed = gt_references - parsed_references
    extra_in_parsed = parsed_references - gt_references
    if missing_in_parsed:
        print(f"(!) References missing in parsed data: {missing_in_parsed}")
        for ref in missing_in_parsed:
            missing_row = ground_truth_df[ground_truth_df['reference'].str.strip() == ref]
            print(f"    Missing transaction details: {missing_row.to_dict(orient='records')}")
    if extra_in_parsed:
        print(f"(!) Extra references in parsed data: {extra_in_parsed}")
        for ref in extra_in_parsed:
            extra_row = parsed_df[parsed_df['reference'].str.strip() == ref]
            print(f"    Extra transaction details: {extra_row.to_dict(orient='records')}")

    # Only proceed to detailed comparison if references match
    if not missing_in_parsed and not extra_in_parsed:
        print("(*) All references match between ground truth and parsed data.")
        # join ground_truth_df and parsed_df on reference column
        merged_df = pd.merge(ground_truth_df, parsed_df, on=["reference"], suffixes=('_gt', '_parsed'))

        # reorder columns for better readability
        columns_order = [
            "reference",
            "transaction_date_gt", "transaction_date_parsed",
            "posting_date_gt", "posting_date_parsed", 
            "description_gt", "description_parsed",
            "mcc_gt", "mcc_parsed",
            "charge_gt", "charge_parsed"
        ]
        merged_df = merged_df[columns_order]
        print("(*) Joined DataFrame for detailed comparison:")
        display(merged_df.head())

        # check for discrepancies in each field
        # Calculate accuracy for specified columns
        accuracy_results = {}
        columns_for_comparison = ["transaction_date", "posting_date", "description", "mcc", "charge"]

        for column in columns_for_comparison:
            correct_count = np.sum(merged_df[f"{column}_gt"].str.lower().str.strip() == merged_df[f"{column}_parsed"].str.lower().str.strip())
            total_count = len(merged_df)
            accuracy = correct_count / total_count if total_count > 0 else 0
            accuracy_results[column] = accuracy

        # Output accuracy results
        print("\nAccuracy Results:")
        for col, acc in accuracy_results.items():
            print(f"{col}: {acc:.2%}")

        # Identify and output unmatched rows
        unmatched_mask = pd.Series(False, index=merged_df.index)
        for col in columns_for_comparison:
            # Create mask for current column mismatch
            mismatch = merged_df[f"{col}_gt"].str.lower().str.strip() != merged_df[f"{col}_parsed"].str.lower().str.strip()
            unmatched_mask |= mismatch

        unmatched_rows = merged_df[unmatched_mask]

        if merged_df.empty:
            print("\nMerged DataFrame is empty - no data to compare.")
        else:
            if not unmatched_rows.empty:
                print("\nUnmatched Rows (showing ground truth vs parsed values):")
                
                # Create display columns (only show mismatched columns)
                display_columns = []
                for col in columns_for_comparison:
                    display_columns.append(f"{col}_gt")
                    display_columns.append(f"{col}_parsed")
                
                # Format output for readability
                unmatched_display = unmatched_rows[display_columns].reset_index(drop=True)
                unmatched_display.columns = [f"{col} (gt)" for col in columns_for_comparison] + [f"{col} (parsed)" for col in columns_for_comparison]
                
                # Print with consistent formatting
                print(unmatched_display.to_string(index=False, justify='left'))
            else:
                print("\nNo unmatched rows found - all values match ground truth")



Ground Truth Count: 30, Parsed Count: 30
(*) Transaction counts match between ground truth and parsed data.
(*) All references match between ground truth and parsed data.
(*) Joined DataFrame for detailed comparison:


,reference,transaction_date_gt,transaction_date_parsed,posting_date_gt,posting_date_parsed,description_gt,description_parsed,mcc_gt,mcc_parsed,charge_gt,charge_parsed
0,24692163316103100975838,11/12,11/12,11/13,11/13,AMZN Mktp US*WB9OW3RL3 Amzn.com/billWA,AMZN Mktp US*WB9OW3RL3 Amzn.com/billWA,5942,5942,169.72,169.72
1,24431063318700451520893,11/14,11/14,11/15,11/15,TOP GOLF LOUISVILLE 64-3 866-867-4653 KY,TOP GOLF LOUISVILLE 64-3 866-867-4653 KY,7999,7999,112.00,112.00
2,24445713319300361413611,11/15,11/15,11/16,11/16,KROGER #356 MIDDLETOWN KY,KROGER #356 MIDDLETOWN KY,5411,5411,68.05,68.05
3,24009583319300746515036,11/15,11/15,11/16,11/16,CKITCHEN COM 718-338-5800 NY,CKITCHEN COM 718-338-5800 NY,5046,5046,1827.00,1827.00
4,24445713320300357682895,11/16,11/16,11/17,11/17,KROGER #356 MIDDLETOWN KY,KROGER #356 MIDDLETOWN KY,5411,5411,83.89,83.89



Accuracy Results:
transaction_date: 100.00%
posting_date: 100.00%
description: 100.00%
mcc: 100.00%
charge: 100.00%

No unmatched rows found - all values match ground truth
